In [1]:
import pandas as pd
import numpy as np
import dash
import flask
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from pandas_datareader import data as web
import datetime as dt
import pymongo
import datetime

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
password = os.getenv("PASSWORD")
username = os.getenv("USERNAME")
location = os.getenv("LOCATION")
collection = os.getenv("COLLECTION")

In [23]:
location

'27017/1grams'

In [24]:
collection

'en'

In [54]:
client = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))
db = client[location[6:]]

In [63]:
query="christmas"

In [73]:
output=dict()
output['word']=query
# Build df
df = pd.DataFrame(list(db[collection].find({ "word": query})));
df=df.dropna(how='all')
df = df.sort_values(by=['time'])
df['year'] = [date.year for date in df['time']]
df['day'] = [date.timetuple().tm_yday for date in df['time']]
# Pull out beginning part of date (YY-MM-DD)
df['time'] = [str(t)[:10] for t in df['time']]

In [ ]:
# Convert date to date object
#print("Before converting to date object, df.head(10) = ")
#print(df.head(10))
df['time'] = [dt.datetime.strptime(t, '%Y-%m-%d').date() for t in df['time']]
# Remove dates before 2010
#print("Before removing early dates, shape is", df.shape)
df=df[df['time']>=(dt.date(2009,8,1))]
#print("After removing early dates, shape is", df.shape)

In [ ]:
# Calculate min, max, and median rank
output['maxrank']=int(min(df['rank'].values))
output['minrank']=int(max(df['rank'].values))
output['medianrank']=int(np.round(np.median(df['rank'].values)))

In [ ]:
print(output)

In [ ]:
# Index df by date
#print("setting time as index...")
df.set_index('time',inplace=True)
#print("set time as index")

In [71]:
# Find today's date
dt.datetime.today().date()
# How many days should we have in our dataset?
daterange=(dt.datetime.today().date()-df.index.min()).days
# Which days are we missing data for?
# Get a list of all days between today and the beginning of time...
missingdays=[(dt.datetime.today() - dt.timedelta(days=x+1)).date() for x in range(daterange)]
# ... then remove any days that we *do* have in our data from our list of missing dates:
original_shape=df.shape
for day in df.index:
    try:
        missingdays.remove(day)
        #print("removed ",day)
    except:
        continue
print("Before adding entries for missing days, df.shape = ",original_shape)
print("After adding entries for missing days, df.shape = ",df.shape)
print("Added ",df.shape[0]-original_shape[0]," new entries")

After adding entries for missing days, df.shape =  (3628, 7)


In [21]:
# Fill any missing days with placeholder values
print("filling missing days...")
for day in missingdays:
    df.loc[day] = None
    df.loc[day]['counts'] = int(0)
    df.loc[day]['rank'] = int(999999999)
    df.loc[day]['freq'] = float(0.0)
#print("np.nan = "+df.loc[df['rank']==np.nan])
#print("None = "+df.loc[df['rank']==None])
print("filled missing days")
print("After filling missing days, df.head(10)=")
df.tail(10)

filling missing days...


/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



filled missing days
After filling missing days, df.head(10)=


,_id,counts,freq,rank,word,year,day
time,,,,,,,
2019-07-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-04-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
head(10) = ")
print(df.head(10))
# Sort days
print("sorting by date...")
df.sort_values(by='time',ascending=True,inplace=True)
print("sorted by date")
# Convert time back to a string
df.index=[t.strftime("%Y-%m-%d") for t in df.index]
# Find the date on which the rank reached its first and most recent (could be the same day) minimum and maximum 
output['firstmax']=df[df['rank']==output['maxrank']].index[-1]
output['lastmax']=df[df['rank']==output['maxrank']].index[0]
output['firstmin']=df[df['rank']==output['minrank']].index[-1]
output['lastmin']=df[df['rank']==output['minrank']].index[0]
# Send dates, ranks and counts as arrays to the output dict
output['times']=df.index.values.tolist()
output['ranks']=[int(r) for r in df['rank'].values] # Convert from int64 to Python integers
output['counts']=[int(c) for c in df['counts'].values] # Convert from int64 to Python integers
output['freq']=[float(f) for f in df['freq'].values] # Convert from float64 Python float
# Send the object to a JSON file
url = '../ui/data/'+query+'.json'
output

In [79]:
loadobject('christmas')

Before converting to date object, df.head(10) = 
                          _id  counts      freq    rank        time  \
135  5d36a36f7ad7ce04178e1cc9       1  0.001062   327.5  2006-07-01   
145  5d36a39a7ad7ce0417968872       1  0.001129   296.0  2006-08-12   
123  5d36a3397ad7ce0417847588       1  0.000247  1003.0  2006-09-19   
17   5d369fa67ad7ce041cce628a       1  0.000364   710.0  2006-10-09   
100  5d36a2f77ad7ce041778a58b       1  0.000338   792.5  2006-10-21   
107  5d36a3027ad7ce04177a6e9e       1  0.000234  1049.0  2006-10-29   
119  5d36a3337ad7ce0417837630       2  0.000356   444.0  2006-11-03   
129  5d36a33e7ad7ce0417855129       1  0.000095  2053.0  2006-11-08   
130  5d36a3407ad7ce041785aa86       2  0.000227   630.5  2006-11-09   
147  5d36a39b7ad7ce041796b1a0       1  0.000097  2063.5  2006-11-14   

          word  year  day  
135  christmas  2006  182  
145  christmas  2006  224  
123  christmas  2006  262  
17   christmas  2006  282  
100  christmas  2006  294  
1

/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



filled missing days
After filling missing days, df.head(10) = 
                                 _id  counts      freq    rank       word  \
time                                                                        
2009-08-01  5d36bf5b7ad7ce0429ce00ea   150.0  0.000009  6998.0  christmas   
2009-08-02  5d36c31f7ad7ce042973063b    53.0  0.000007  8222.5  christmas   
2009-08-06  5d36c54e7ad7ce0429d357d0    36.0  0.000007  8496.0  christmas   
2009-08-07  5d36c5a77ad7ce0429e2066e   112.0  0.000007  8482.0  christmas   
2009-08-08  5d36c8237ad7ce04294c99f0   115.0  0.000008  7685.5  christmas   
2009-08-09  5d36c9297ad7ce042979ef5a   185.0  0.000009  6510.5  christmas   
2009-08-10  5d36cb597ad7ce0429d947fe   185.0  0.000008  7439.0  christmas   
2009-08-11  5d36cdd07ad7ce042945e8f3   158.0  0.000007  8194.0  christmas   
2009-08-12  5d36cf607ad7ce04298a6458   192.0  0.000008  7569.0  christmas   
2009-08-13  5d36d1a37ad7ce0429ecfcd7   171.0  0.000007  8152.5  christmas   

            

ValueError: cannot convert float NaN to integer

# Input box for query

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

data=[]

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    dcc.Input(id='my-id', value='@realdonaldtrump', type='text'),
    dcc.Graph(id='my-graph',figure={'layout': {'yaxis': {'autorange': 'reversed', 'type':'log'}}}),
])

@app.callback(Output('my-graph', 'figure'), [Input('my-id', 'value')])
def update_graph(input_value):
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    input_value = input_value.replace({"   ":",","  ":","," ":",",",,":",",",,,":","})
    values = input_value.split(',')
    for item in values:
        item = lower(item)
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure

if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0')

# Dropdown for query

### Load wordlist

In [ ]:
import csv
wordlist = []
with open('wordlist_from-df.csv', 'rt') as f:
    reader = csv.reader(f)
    next(reader)
    wordlist=list(reader)

In [ ]:
wordDF=pd.DataFrame(wordlist, columns=['word','rank'])

In [ ]:
wordDF=wordDF[wordDF['rank']!='Rank']

In [ ]:
wordDF=wordDF[wordDF['rank']!='minRank']

In [ ]:
wordDF['rank']=wordDF['rank'].astype(int);

### Dropdown with under 5k

In [ ]:
wordDF[wordDF['rank']<5000].shape

In [ ]:
options = []
wordDF_filtered = wordDF[wordDF['rank']<5000]
for word in wordDF_filtered['word']:
    options.append({'label': word,'value': word})

In [ ]:
options[:5]

In [ ]:
options[100:105]

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    html.P('This shows any one-grams that cracked the top 5k rank at any time (47,844 onegrams)'),
    html.Div([
        dcc.Dropdown(id='my-dropdown', options=options, value=['@realdonaldtrump','@bts_twt'], multi=True)]),
    dcc.Graph(id='my-graph',figure={'layout': {'yaxis': {'autorange': 'reversed', 'type':'log'}}})
])

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])

def update_graph(input_value):
    data=[]
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    input_value = input_value.replace({"   ":",","  ":","," ":",",",,":",",",,,":","})
    for item in input_value:
        item = lower(item)
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0')

### Dropdown of under 1k list, with ability to add options

In [ ]:
options = []
wordDF_filtered = wordDF[wordDF['rank']<1000]
wordDF_filtered=wordDF_filtered.sort_values(by='rank')
for word in wordDF_filtered['word']:
    options.append(word)

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    html.P('Select a one-gram that cracked the top 1k rank at any time (1,056 onegrams)'),
    html.Datalist(id="wordlist", children=[]),
    dcc.Input(
        id='input-field',
        name='symbol',
        list='wordlist',
        placeholder='Enter a value...',
        type='text',
        value='trump'
    ),
    dcc.Graph(id='my-graph',figure={'layout': {'yaxis': {'autorange': 'reversed', 'type':'log'}}})
])

@app.callback(Output('wordlist', 'children'),[Input('input-field', 'value')])
def callback(item):
    if len(item) < 5:
        wordDF_filtered = wordDF[wordDF['rank']<1000]
    else:
        wordDF_filtered = wordDF[wordDF['word'].str.contains(item)]
    for word in wordDF_filtered:
        options.append(word)
    return options

@app.callback(Output('my-graph', 'figure'), [Input('input-field', 'value')])
def update_graph(input_value):
    data=[]
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    input_value = input_value.replace({"   ":",","  ":","," ":",",",,":",",",,,":","})
    values = input_value.split(',')
    for item in values:
        item = lower(item)
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0')

### Dynamic Dropdown (abandoned)

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    html.P('Select a one-gram that cracked the top 5k rank at any time (47,844 onegrams)'),
    html.Div([
        dcc.Dropdown(id='my-dropdown', options=options, value=['@realdonaldtrump','@bts_twt'], multi=True)]),
    dcc.Graph(id='my-graph',figure={'layout': {'yaxis': {'autorange': 'reversed', 'type':'log'}}})
])

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])

def update_graph(input_value):
    data=[]
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    for item in input_value:
        item = lower(item)
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure

@app.callback(Output('my-dropdown', 'options'), [Input('my-dropdown', 'value')])

def update_options(input_value):
    for item in input_value:
        if len(item) < 5:
            wordDF_filtered = wordDF[wordDF['rank']<1000]
        else:
            wordDF_filtered = wordDF[wordDF['word'].str.contains(item)]
        for word in wordDF_filtered:
            options.append({'label': word[0],'value': word[0]})
    return options


if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0')